In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk


In [2]:
#convert all 
data_2014 = pd.read_csv("./2014_Financial_Data.csv")
data_2015 = pd.read_csv("./2015_Financial_Data.csv")
data_2016 = pd.read_csv("./2016_Financial_Data.csv")
data_2017 = pd.read_csv("./2017_Financial_Data.csv")
data_2018 = pd.read_csv("./2018_Financial_Data.csv")


In [3]:
#the companies are denoted by their stock ticker but the ticker column is unnamed
#this cell renames that to "Ticker" for all dataframes
data_2014.rename( columns={'Unnamed: 0':'Ticker'}, inplace=True )
data_2015.rename( columns={'Unnamed: 0':'Ticker'}, inplace=True )
data_2016.rename( columns={'Unnamed: 0':'Ticker'}, inplace=True )
data_2017.rename( columns={'Unnamed: 0':'Ticker'}, inplace=True )
data_2018.rename( columns={'Unnamed: 0':'Ticker'}, inplace=True )


In [4]:
#construct 5 different sets of ticker labels, 1 for eachh data frame
ticker_set_2014 = set(data_2014["Ticker"])
ticker_set_2015 = set(data_2015["Ticker"])
ticker_set_2016 = set(data_2016["Ticker"])
ticker_set_2017 = set(data_2017["Ticker"])
ticker_set_2018 = set(data_2018["Ticker"])
#find the intersection of the sets. This limits us to those companies for whom we have data from all five years
all_common_companies = list(set.intersection(ticker_set_2014, ticker_set_2015, ticker_set_2016, ticker_set_2017,
                                        ticker_set_2018))
#filter dataframes using the list of common companies we created above
data_2014 = data_2014[data_2014["Ticker"].isin(all_common_companies)]
data_2015 = data_2015[data_2015["Ticker"].isin(all_common_companies)]
data_2016 = data_2016[data_2016["Ticker"].isin(all_common_companies)]
data_2017 = data_2017[data_2017["Ticker"].isin(all_common_companies)]
data_2018 = data_2018[data_2018["Ticker"].isin(all_common_companies)]

In [10]:

#Alright, lets try to get out the data we want from the 2014 array and make a new Pandas data frame with it

# Removed "Ticker", from the first entry of each of these
new_2014 = data_2014[['EPS','EBIT Margin', 'Net Profit Margin','Net Cash/Marketcap','priceBookValueRatio','priceEarningsRatio', 'priceCashFlowRatio',
 'returnOnAssets','inventoryTurnover','cashFlowToDebtRatio','totalDebtToCapitalization','Earnings Yield','Debt to Assets', 'Current ratio','Graham Net-Net']]
labels_2014 = data_2014[['Class']]
new_2015 = data_2015[['EPS','EBIT Margin', 'Net Profit Margin','Net Cash/Marketcap','priceBookValueRatio','priceEarningsRatio', 'priceCashFlowRatio',
 'returnOnAssets','inventoryTurnover','cashFlowToDebtRatio','totalDebtToCapitalization','Earnings Yield','Debt to Assets', 'Current ratio','Graham Net-Net']]
labels_2015 = data_2015[['Class']]
new_2016 = data_2016[['EPS','EBIT Margin', 'Net Profit Margin','Net Cash/Marketcap','priceBookValueRatio','priceEarningsRatio', 'priceCashFlowRatio',
 'returnOnAssets','inventoryTurnover','cashFlowToDebtRatio','totalDebtToCapitalization','Earnings Yield','Debt to Assets', 'Current ratio','Graham Net-Net']]
labels_2016 = data_2016[['Class']]
new_2017 = data_2017[['EPS','EBIT Margin', 'Net Profit Margin','Net Cash/Marketcap','priceBookValueRatio','priceEarningsRatio', 'priceCashFlowRatio',
 'returnOnAssets','inventoryTurnover','cashFlowToDebtRatio','totalDebtToCapitalization','Earnings Yield','Debt to Assets', 'Current ratio','Graham Net-Net']]
labels_2017 = data_2017[['Class']]
new_2018 = data_2018[['EPS','EBIT Margin', 'Net Profit Margin','Net Cash/Marketcap','priceBookValueRatio','priceEarningsRatio', 'priceCashFlowRatio',
 'returnOnAssets','inventoryTurnover','cashFlowToDebtRatio','totalDebtToCapitalization','Earnings Yield','Debt to Assets', 'Current ratio','Graham Net-Net']]
labels_2018 = data_2018[['Class']] # CHANGED THIS TO 2018


In [11]:
#Defining the max number of NaN to data points
nanRatio = 0.1


#Now I am going to try to itter over the rows and figure out how many NaNs we have 
dataPoints2014 = new_2014.shape[0]
dataPoints2015 = new_2015.shape[0]
dataPoints2016 = new_2016.shape[0]
dataPoints2017 = new_2017.shape[0]
dataPoints2018 = new_2018.shape[0]
#Now I am going to try to itter over the rows and figure out how many NaNs we have 
dataPoints2014 = new_2014.shape[0]
dataPoints2015 = new_2015.shape[0]
dataPoints2016 = new_2016.shape[0]
dataPoints2017 = new_2017.shape[0]
dataPoints2018 = new_2018.shape[0]

null_2014 = new_2014.isnull().sum(axis=0)
null_2015 = new_2015.isnull().sum(axis=0)
null_2016 = new_2016.isnull().sum(axis=0)
null_2017 = new_2017.isnull().sum(axis=0)
null_2018 = new_2018.isnull().sum(axis=0)
#My Plan is to delete rows with NaN but If the majority rows have NaN then 
#it would be better to delete the column 
#making a ratio of NaN to data Points to mask with
null_2014 = null_2014 / dataPoints2014
null_2015 = null_2015 / dataPoints2015
null_2016 = null_2016 / dataPoints2016
null_2017 = null_2017 / dataPoints2017
null_2018 = null_2018 / dataPoints2018
#Things with high Null Ratio
#Current Ratio 0.25
#cashFlowToDebtRatio 0.41
#return on assets 0.25
#Price Cash Flor 0.25
#Price Booking 0.16

In [40]:
#Gonna get out the feature with the least Nan ratio
nullItter = null_2014.sort_values(axis = 0)
#Now need to get the column label
colLabel = nullItter.index
len = colLabel.size

from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA # Added this
from sklearn.model_selection import train_test_split # Added this import for splitting training and testing data
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

scalor = StandardScaler()
#Gonna start with the column with the least nans and for each itteration add
#another feature
totErrorSVM = np.zeros([len-1]) 
totErrorNN = np.zeros([len-1]) 
#mySvm = svm.SVC() # kernel = rbf. rbf: 0.589095744680851.
#mySvm = svm.SVC(kernel='linear') # kernel = linear. linear: 0.5897606382978723
mySvm = svm.SVC(kernel='poly') # kernel = poly. poly: 0.5948581560283688
#mySvm = svm.SVC(kernel='sigmoid') # kernel = sigmoid. sigmoid: 0.5518617021276596
#mySvm = svm.SVC(kernel='precomputed') # kernel = precomputed. precomputed: error
#mySvm = svm.SVC(kernel='rbf') # kernel = rbf. rbf: 0.589095744680851
#clf = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(100,40), random_state=1) # lbfgs: 0.5806737588652482
clf = MLPClassifier(solver='sgd', alpha=1e-5,hidden_layer_sizes=(100,40), random_state=1) # sgd: 0.5939716312056738
#clf = MLPClassifier(solver='adam', alpha=1e-5,hidden_layer_sizes=(100,40), random_state=1) # adam: 0.5842198581560284
logisticRegr = LogisticRegression(solver='lbfgs') # 0.5966312056737588 accuracy
#logisticRegr = LogisticRegression(solver='newton-cg') # 0.5966312056737588 accuracy
#logisticRegr = LogisticRegression(solver='liblinear') # 0.5957446808510638
#logisticRegr = LogisticRegression(solver='sag') # 0.5948581560283688
#logisticRegr = LogisticRegression(solver='saga') # 0.5948581560283688
mySplitter = KFold(n_splits=5)
#for now, I am going to combine all data sets to make a super data set
totalData = new_2014.append(new_2015,sort=False)
totalData = totalData.append(new_2016,sort=False)
totalData = totalData.append(new_2017,sort=False)
totalData = totalData.append(new_2018,sort=False) # I CHANGED THIS FROM 2017 to 2018!!!
totalLabels = labels_2014.append(labels_2015,sort=False)
totalLabels = totalLabels.append(labels_2016,sort=False)
totalLabels = totalLabels.append(labels_2017,sort=False)
totalLabels = totalLabels.append(labels_2018,sort=False)

# Added this outside the loop to try to feature select only with PCA
myMask = totalData.isnull().sum(axis = 1)
myMask = myMask == 0
myDataSet = totalData[myMask]
myLabels = totalLabels[myMask]

#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.2, random_state=0) # Test 20%: SVM: 0.5904255319148937 accuracy NN: 0.5704787234042553
#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.3, random_state=0) # Test 30%: SVM: 0.5855496453900709 NN: 0.563386524822695
#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.1, random_state=0) # Test 10%: SVM: 0.589095744680851 NN: 0.5731382978723404
trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.15, random_state=0) # Test 15%: SVM: 0.5948581560283688 NN: 0.5806737588652482 
#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.25, random_state=0) # Test 25%: SVM: 0.5867021276595744 NN: 0.575
#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.12, random_state=0) # Test 12%: SVM: 0.592469545957918 NN: 0.5736434108527132
#trainData, testData, trainLabel, testLabel = train_test_split(myDataSet, myLabels, test_size=0.16, random_state=0) # Test 13%: SVM: 0.5930470347648262 NN: 0.5705521472392638
# Test 14%: SVM: 0.5944919278252612 NN: 0.5650522317188984
# Test 16%: SVM: 0.5943474646716542 NN: 0.5810473815461347
#print(trainData)
#scalor.fit(trainData, y=trainLabel) # BETTER FOR NN
scalor.fit(trainData) # BETTER FOR SVM
trainData = scalor.transform(trainData) # BETTER FOR SVM
testData = scalor.transform(testData) # BETTER FOR SVM
#trainData = scalor.fit_transform(trainData, y=trainLabel) # BETTER FOR NN (Basically switch places with accuracy)
#testData = scalor.fit_transform(testData, y=testLabel) # BETTER FOR NN

# Do PCA before using learning methods for more feature selection.
#pca = PCA(0.95) # SVM: 0.5948581560283688 NN: 0.5939716312056738 log reg: 0.5957446808510638
#pca = PCA(0.8) # SVM: 0.5939716312056738 NN: 0.598404255319149 log reg: 0.5966312056737588
#pca = PCA(0.99) # SVM: 0.5948581560283688 NN: 0.6019503546099291 log reg: 0.5966312056737588
pca = PCA(n_components = 13) # 13: 0.5948581560283688, 14: 0.5939716312056738, 12: 0.5948581560283688, 11: 0.5939716312056738, 10: 0.5930851063829787, 9: 0.5939716312056738, 8: 0.5948581560283688, 7: 0.5913120567375887, 6: 0.5930851063829787, 5: 0.5930851063829787, 4: 0.5921985815602837, 3: 0.5913120567375887, 2: 0.5913120567375887, 1: 0.5913120567375887
#pca = PCA(0.999) # Same as above
# No PCA: SVM: 0.5939716312056738 NN: 0.5975177304964538 log reg: 0.5948581560283688
pca.fit(trainData)
trainData = pca.transform(trainData)
testData = pca.transform(testData)

#Fitting an SVM and Neural Network to the training data set 
myModel = mySvm.fit(trainData,trainLabel)
clf.fit(trainData,trainLabel)
logisticRegr.fit(trainData, trainLabel)
#Predicting the Test Data 
myTestSVM_predict = mySvm.predict(testData)
myTestNN_predict = clf.predict(testData)
logisticRegr_predict = logisticRegr.predict(testData)
print("Logistic Regression Accuracy", logisticRegr.score(testData, testLabel))
#print(metrics.accuracy_score(testLabel, logisticRegr_predict))
print("SVM Accuracy", metrics.accuracy_score(testLabel, myTestSVM_predict))
print("NN Accuracy", metrics.accuracy_score(testLabel, myTestNN_predict))
"""
for i in range(len-1):
    #Getting out how many labels we want 
    myFeat = colLabel[1:i+2]
    myDataSet = totalData[myFeat]
    #Now need to remove the rows with NaN 
    myMask = myDataSet.isnull().sum(axis = 1)
    myMask = myMask == 0
    myDataSet = myDataSet[myMask]
    myLabels = totalLabels[myMask]
    foldErrorSVM = 0
    foldErrorNN = 0
    #Gonna split the data into K folds and train an svm model 
    for trainIdx,testIdx in mySplitter.split(myDataSet):
        #Note convert your pandas data frame into a numpy array 
        #
        trainData = myDataSet.iloc[trainIdx]
        trainData = trainData.to_numpy()
        
    #
        testData = myDataSet.iloc[testIdx]
        testData = testData.to_numpy()
        trainLabel = myLabels.iloc[trainIdx]
        trainLabel = trainLabel.to_numpy()[:,0]
        testLabel = myLabels.iloc[testIdx]
        testLabel = testLabel.to_numpy()[:,0]
        
        #Need to standardize data set 
        #trainPoints = trainData.shape[0]
        #trainMean = np.sum(trainData,axis = 0) / trainPoints
        #trainData = trainData - trainMean
    
        scalor.fit(trainData)
        trainData = scalor.transform(trainData)
        testData = scalor.transform(testData)
        
        # Do PCA before using learning methods for more feature selection.
        #pca = PCA(0.95)
        #pca.fit(trainData)
        #trainData = pca.transform(trainData)
        #testData = pca.transform(testData)

        #Fitting an SVM and Neural Network to the training data set 
        myModel = mySvm.fit(trainData,trainLabel)
        clf.fit(trainData,trainLabel)
        #Predicting the Test Data 
        myTestSVM = mySvm.predict(testData)
        myTestNN = clf.predict(testData)
        #Using Euclidean distance to get the fold error
        myErrorSVM = myTestSVM == testLabel
        myErrorNN = myTestNN == testLabel
        myErrorSVM = np.sum(myErrorSVM)
        myErrorNN = np.sum(myErrorNN)
        #note we need to divide by the number of data points each time because each loop 
        #the number of data points is getting smaller 
        foldErrorSVM = foldErrorSVM + (myErrorSVM/ trainData.shape[0])
        foldErrorNN = foldErrorNN + (myErrorNN/ trainData.shape[0])
    totErrorSVM[i]= foldErrorSVM
    totErrorNN[i] = foldErrorNN

#Lets plot how the error changes
plt.plot(range(len-1),totErrorSVM,'*')
plt.plot(range(len-1),totErrorNN,'g*')
plt.xlabel('Number of Features')
plt.ylabel('accuracy ')
plt.legend(['SVM', 'Neural Network'])
plt.show()
"""


C:\Users\brice\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\brice\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\brice\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\brice\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


Logistic Regression Accuracy 0.5948581560283688
SVM Accuracy 0.5948581560283688
NN Accuracy 0.6019503546099291


"\nfor i in range(len-1):\n    #Getting out how many labels we want \n    myFeat = colLabel[1:i+2]\n    myDataSet = totalData[myFeat]\n    #Now need to remove the rows with NaN \n    myMask = myDataSet.isnull().sum(axis = 1)\n    myMask = myMask == 0\n    myDataSet = myDataSet[myMask]\n    myLabels = totalLabels[myMask]\n    foldErrorSVM = 0\n    foldErrorNN = 0\n    #Gonna split the data into K folds and train an svm model \n    for trainIdx,testIdx in mySplitter.split(myDataSet):\n        #Note convert your pandas data frame into a numpy array \n        #\n        trainData = myDataSet.iloc[trainIdx]\n        trainData = trainData.to_numpy()\n        \n    #\n        testData = myDataSet.iloc[testIdx]\n        testData = testData.to_numpy()\n        trainLabel = myLabels.iloc[trainIdx]\n        trainLabel = trainLabel.to_numpy()[:,0]\n        testLabel = myLabels.iloc[testIdx]\n        testLabel = testLabel.to_numpy()[:,0]\n        \n        #Need to standardize data set \n        #t